In [1]:
from tkinter.filedialog import askopenfilename
from tkinter import messagebox 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from seleniumbase import Driver
from selenium.common.exceptions import NoSuchWindowException
from datetime import datetime
import re
import urllib
import os
import time    
import pandas as pd
import tkinter as tk

In [2]:
class AutomacaoWhatsApp:
    @staticmethod
    def data():
        data = datetime.now()
        return f"{data.day}{data.month}{data.year}"
    def __init__(self, window):
        self.window = window
        self.window.geometry("946x606")
        self.window.configure(bg="#249e98")
        self.window.title('Automação de envio de mensagem no WhatsApp')
        self.canvas = tk.Canvas(
            self.window,
            bg="#249e98",
            height=606,
            width=946,
            bd=0,
            highlightthickness=0,
            relief="ridge")
        self.canvas.place(x=0, y=0)

        self.background_img = tk.PhotoImage(file=fr"imagens/background.png")
        self.background = self.canvas.create_image(
            309.0, 309.5,
            image=self.background_img)

        self.canvas.create_text(
            534.5, 90.5,
            text="Arquivo",
            fill="#fefefe",
            font=("Lusitana", int(18.0)))

        self.canvas.create_text(
            526.0, 228.5,
            text="Texto",
            fill="#fefefe",
            font=("Lusitana", int(18.0)))

        self.img0 = tk.PhotoImage(file=fr"imagens/img0.png")
        self.b0 = tk.Button(
            self.canvas,
            image=self.img0,
            borderwidth=0,
            highlightthickness=0,
            command=self.automacao,
            relief="flat")
        self.b0.place(
            x=716, y=513,
            width=187,
            height=59)

        self.caixa_texto_img = tk.PhotoImage(file=fr"imagens/img_textBox0.png")
        self.caixa_texto_bg = self.canvas.create_image(
            703.0, 350.0,
            image=self.caixa_texto_img)

        self.caixa_texto = tk.Text(
            self.canvas,
            bd=0,
            bg="#fefefe",
            highlightthickness=0)
        self.caixa_texto.place(
            x=503, y=251,
            width=400,
            height=196)

        self.botao_arquivo_img = tk.PhotoImage(file=fr"imagens/img_textBox1.png")
        self.botao_arquivo_bg = self.canvas.create_image(
            703.0, 137.0,
            image=self.botao_arquivo_img)

        self.var_caminho = tk.StringVar()
        self.botao_arquivo = tk.Button(
            self.canvas,
            bd=0,
            bg="#fefefe",
            highlightthickness=0,
            text='',
            command=self.abrir_arquivo)
        self.botao_arquivo.place(
            x=503, y=112,
            width=400,
            height=48)

        self.window.resizable(False, False)

    def abrir_arquivo(self):
        caminho = askopenfilename(title='Escolha o arquivo excel',
                                              initialdir=os.getcwd(),
                                              filetypes=[("Excel Files", "*.xlsx")])
        if len(caminho) == 0:
            return
        else:
            self.var_caminho.set(caminho)
            self.botao_arquivo['text'] = self.var_caminho.get()

    def automacao(self):
        resposta = messagebox.askquestion("Confirmação", "Deseja inicializar a automação?")
        if resposta == 'yes' and len(self.var_caminho.get()) > 0:
            try:
                msg_inicial = self.caixa_texto.get('1.0', tk.END)
                df, lista_colunas = self.tratar_dataframe()
                messagebox.showinfo("Informação", "Escaneie o QR Code do navegador para inicializar o envio")
                nav = Driver()
                nav.get('https://web.whatsapp.com/')
                nav.maximize_window()
                while len(nav.find_elements(By.XPATH, '//*[@id="side"]/div[1]/div/div[2]')) == 0:
                    time.sleep(0.1)
                for linha in df.index:
                    mensagem = msg_inicial
                    numero_celular = None
                    for coluna in lista_colunas:
                        valor = df.loc[linha, coluna]
                        if numero_celular == None:
                            numero_celular = self.encontrar_numero(valor)
                        mensagem = self.criar_mensagem(lista_colunas, coluna, mensagem, valor)
                    if numero_celular == None:
                        df.loc[linha, "Status Envio"] = "Sem Número"
                        continue
                    nav.get(f"https://web.whatsapp.com/send?phone={numero_celular}&text={mensagem}")
                    while len(nav.find_elements(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[2]')) == 0:
                        time.sleep(1)
                    time.sleep(5)
                    nav.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[1]/p').send_keys(Keys.ENTER)
                    df.loc[linha, "Status Envio"] = "Enviado"
                    time.sleep(10)
            except NoSuchWindowException:
                messagebox.showwarning("Alerta", "Automação Interrompida")
            except:
                pass
            else:
                nav.close()
                messagebox.showinfo("Informação", "Automatização Finalizada")
            finally:
                data_atual = AutomacaoWhatsApp.data()
                self.criar_excel(data_atual, df)
        elif resposta == 'No':
            return
        elif len(self.var_caminho.get()) == 0 and resposta == 'yes':
            messagebox.showerror("Erro", "Preencha os campos corretamente e tente novamente.")

    def tratar_dataframe(self):
        df = pd.read_excel(self.var_caminho.get())
        lista_colunas = df.columns
        df["Status Envio"] = "Não Enviado"
        df = df.astype(str)
        return df, lista_colunas

    def criar_mensagem(self, lista, coluna, msg, valor):
        placeholder = fr"<<{coluna}>>"
        if placeholder in msg:
            msg = msg.replace(placeholder, valor)
        if coluna == lista[-1]:
            msg = urllib.parse.quote(msg)
        return msg

    def encontrar_numero(self, valor):
        padrao = re.compile(r'\+\d+')
        num = re.search(padrao, valor)
        if num:
            numero = num.group(0)
            numero = numero.replace('"', '').replace(' ', '').replace('-', '')
            return numero

    def criar_excel(self, data, df):
        caminho = os.getcwd()
        caminho_log = caminho + '\logs'
        if not os.path.isdir(caminho_log):
            os.mkdir(caminho_log)
        arquivo = caminho_log + f'\Status{data}.xlsx'
        df.to_excel(arquivo, index=False)
 

In [3]:
if __name__ == "__main__":
    janela = tk.Tk()
    app = AutomacaoWhatsApp(janela)
    janela.mainloop()